## 딕셔너리(json) <-> 판다스 df, 딕셔너리,판다스 df <-> qdrant 명심하자

In [ ]:
from datasets import load_dataset
import json
import pandas as pd

dataset = json.load(open("/Users/js/tasteam-aicode-gpu-all-python-process-runtime_for_github/data/test_data_sample.json","r"))
df = pd.DataFrame(dataset)
df.head()

,id,restaurant_id,restaurant_name,food_category_id,food_category_name,member_id,group_id,subgroup_id,content,is_recommended,created_at,updated_at,deleted_at,images
0,1,1,Test Restaurant 1,1,한식,2,None,None,맛이쪙 꼬소 행,True,2025-02-17T17:02:45.366789,None,None,[]
1,2,1,Test Restaurant 1,1,한식,3,None,None,4시쯤 가서 예약하고 6시 20분에 가져왔어요 먹어도 먹어도 안 질려요,True,2025-11-14T17:02:45.367453,None,None,[]
2,3,1,Test Restaurant 1,1,한식,4,None,None,별내에 있다는 게 전혀 어울리지 않는 수제 타르 트집!! 파리 14구쯤에 있다면 어...,True,2025-12-29T17:02:45.367665,None,None,[]
3,4,1,Test Restaurant 1,1,한식,5,None,None,분위기도 좋고 갈비가 진짜 끝내줘요 룸이 있어서 조용히 식사하기도 최고예요,True,2025-08-23T17:02:45.367871,None,None,[]
4,5,1,Test Restaurant 1,1,한식,6,None,None,굿 좋은 가격 좋은 맛,True,2025-09-07T17:02:45.368073,None,None,[]


## qdrant 시작

In [120]:
from fastembed import TextEmbedding, SparseTextEmbedding

dense_model = TextEmbedding('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
dense_embeddings = list(dense_model.embed(df["content"]))

sparse_model = SparseTextEmbedding('Qdrant/bm25')
sparse_embeddings = list(sparse_model.embed(df["content"]))

/var/folders/n7/239xybp17cz6h8slgxrclly40000gn/T/ipykernel_52654/3326046019.py:3: UserWarning: The model sentence-transformers/paraphrase-multilingual-mpnet-base-v2 now uses mean pooling instead of CLS embedding. In order to preserve the previous behaviour, consider either pinning fastembed version to 0.5.1 or using `add_custom_model` functionality.
  dense_model = TextEmbedding('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')


In [121]:
from qdrant_client import QdrantClient, models
from qdrant_client.models import VectorParams, Distance, SparseVectorParams, PointStruct, SparseVector

def setup_qdrant_hybrid(passages, dense_embeddings, sparse_embeddings):
    client = QdrantClient('http://localhost:6333')
    collection_name = "hybrid_test"
    dense_dim = len(dense_embeddings[0])
    
    if client.collection_exists(collection_name):
        client.delete_collection(collection_name)
        
    client.create_collection(
        collection_name = collection_name,
        vectors_config = {
            "dense" : VectorParams(
                size = dense_dim,
                distance = Distance.COSINE
            )
        },
        sparse_vectors_config = {
            "sparse" : SparseVectorParams()
        }
    )
    
    points = []
    for i, (passage, dense_emb, sparse_emb) in enumerate(zip(passages, dense_embeddings, sparse_embeddings)):
        
        dense_vector = list(dense_emb)
    
        sparse_vector = SparseVector(
            indices = list(sparse_emb.indices),
            values = list(sparse_emb.values)
        )
        
        point = PointStruct(
            id = i,
            vector = {
                "dense" : dense_vector,
                "sparse" : sparse_vector
            },
            payload = passages[i] # 딕서너리 -> qdrant 가능, 딕셔너리 <-> 판다스 df, 판다스 df -> qdrant payload 가능
        )
        points.append(point)
        
    client.upsert(collection_name=collection_name, points=points)
    return client

In [122]:
client = setup_qdrant_hybrid(dataset, dense_embeddings, sparse_embeddings)

In [123]:
from qdrant_client import models

def query_qdrant_hybrid(
    client,
    query_dense_emb,    # list[float] or np.ndarray
    query_sparse_emb,   # fastembed SparseEmbedding (indices, values)
    limit=5,
):
    dense_vec = list(query_dense_emb)
    sparse_vec = models.SparseVector(
        indices=list(query_sparse_emb.indices),
        values=list(query_sparse_emb.values),
    )

    res = client.query_points(
        collection_name="hybrid_test",
        query=models.FusionQuery(
            fusion=models.Fusion.RRF
        ),
        prefetch=[
            models.Prefetch(
                query=dense_vec,
                using="dense",
            ),
            models.Prefetch(
                query=sparse_vec,
                using="sparse",
            ),
        ],
        limit=limit,
        with_payload=True,
    )

    return res.points


In [124]:
# 전처리 전, 앞뒤 똑같은 단어 그대로 사용, 카테고리에 어울리지 않는 단어 제거 전
service_seeds = ['직원 친절', '사장 친절', '친절 음식', '친절 기분', '서비스 친절', '친절 서비스', '분위기 서비스', '사장 직원', '친절 사장', '직원 서비스', '친절 고기', '친절 직원', '친절 분위기', '친절 가격', '아주머니 친절', '깔끔 친절']
price_seeds = ['가격 대비', '무한 리필', '가격 생각', '음식 가격', '합리 가격', '생각 가격', '분위기 가격', '메뉴 가격', '가격 만족', '가격 분위기', '가격 퀄리티', '가격 합리', '리필 가능', '커피 가격', '가격 사악', '런치 가격']
food_seeds = ['가락 국수', '평양 냉면', '수제 버거', '크림 치즈', '치즈 케이크', '크림 파스타', '당근 케이크', '오일 파스타', '카페 커피', '비빔 냉면', '커피 원두', '리코타 치즈', '비빔 막국수', '치즈 돈가스', '커피 산미', '치즈 파스타']

seed_list = [service_seeds, price_seeds, food_seeds]
name_list = ["service", "price", "food"]

In [125]:
for seeds, name in zip(seed_list, name_list):

    print(f"\n{name} seeds ({len(seeds)}):")

    # dense query
    dense_q = next(dense_model.embed(" ".join(seeds[:-1])))

    # sparse query
    sparse_q = next(sparse_model.embed(" ".join(seeds[:-1])))

    hits = query_qdrant_hybrid(
        client,
        dense_q,
        sparse_q,
        limit=5
    )

    for h in hits:
        print(h.payload["content"], "score:", h.score)


service seeds (16):
어느 수산시장에서나 나오면 먹을 수 있는 민어.의 맛집  서비스 업종 역사상 최악 평점 별점 17개  기름 전 휴게소 핫바 하나 먹고 카오디오로 음악 크게 틀고 드라이빙하는 것이 이로울 수준.   사유  검열 삭제 score: 0.5
맛있고 친절하세요 score: 0.5
원래도 평양냉면은 별로 즐기는 건 아니지만 여기는 다른 데보 다도 별로였음. 우선 고기 국물이 너무 기름지고, 고기 누린내가 많이 남. 결정적으로 평양냉면 아직 좋아하진 못해도 왠지 생각날 것도 같은 그런 고소한 메밀향이 없음. score: 0.33333334
친절하고 맛난 음식을 먹을 수 있어서 좋아요~ 동료들과 와서 좋은 추억 만들고 갑니다! 맛도 좋아요!! score: 0.33333334
회사 사람들하고 점심 먹으러 왔는데 음식도 맛있고 직원분들이 너무 친절하셔서 기분 좋게 배터지게 먹구 왔습니다 !!!! 특히 크림 카레 너무 부드럽고 취향저격 매일 올꺼같아요!!!!ㅎ score: 0.26666668

price seeds (16):
굿 좋은 가격 좋은 맛 score: 0.5833334
고급 진 초밥 먹을 수 있는 곳! 두꺼운 네타가 올라가있어서 맛있게 먹을 수 있었다. 모둠 초밥 가격대 동네 초밥집 치고 가격대 있는 편이긴 하지만, 네타 상태 보면 비싼 가격대가 이해된다! score: 0.5
얼그레이 케이크는 아직 못 먹어보고, 얼그레이 밀크티만 먹어봤어요 당도 13도 중 2도 먹어봤는데 생각보다 별로 인건 왜죠 밀크티가 조금 연한 거 같아서 아쉬웠네요! 그래도 브런치 세트는 가격 대비 구성이 괜찮은 것 같아요! 밀크티 포함됨 나중엔 얼그레이 케이크와 브런치 다시 먹어볼래요!! score: 0.5
한강진 역에 위치한 카페 울프 소셜클럽. 늑대 그림의 로고가 귀엽지만 울프는 Woolf. 화장실에 쓰인 a restroom of ones own이라는 문구를 보면 더 확실해진다. 음악도 분위기도 모두 좋다. 추천   버터크림 헤븐 라테. 밀크티와 더불어 시

In [126]:
# 전처리 후, 앞뒤 같은 단어 제거, 카테고리에 어울리지 않는 단어 제거
service_seeds = ['직원 친절', '사장 친절', '친절 기분', '서비스 친절', '사장 직원', '직원 서비스', '아주머니 친절']
price_seeds = ['가격 대비', '무한 리필', '가격 생각', '음식 가격', '합리 가격', '메뉴 가격', '가격 만족', '가격 퀄리티', '리필 가능', '커피 가격', '가격 사악', '런치 가격']
food_seeds = ['가락 국수', '평양 냉면', '수제 버거', '크림 치즈', '치즈 케이크', '크림 파스타', '당근 케이크', '오일 파스타', '카페 커피', '비빔 냉면', '커피 원두', '리코타 치즈', '비빔 막국수', '치즈 돈가스', '커피 산미', '치즈 파스타']

seed_list = [service_seeds, price_seeds, food_seeds]
name_list = ["service", "price", "food"]

In [127]:
for seeds, name in zip(seed_list, name_list):

    print(f"\n{name} seeds ({len(seeds)}):")

    # dense query
    dense_q = next(dense_model.embed(" ".join(seeds[:-1])))

    # sparse query
    sparse_q = next(sparse_model.embed(" ".join(seeds[:-1])))

    hits = query_qdrant_hybrid(
        client,
        dense_q,
        sparse_q,
        limit=5
    )

    for h in hits:
        print(h.payload["content"], "score:", h.score)


service seeds (7):
맛있고 친절하세요 score: 0.5
어느 수산시장에서나 나오면 먹을 수 있는 민어.의 맛집  서비스 업종 역사상 최악 평점 별점 17개  기름 전 휴게소 핫바 하나 먹고 카오디오로 음악 크게 틀고 드라이빙하는 것이 이로울 수준.   사유  검열 삭제 score: 0.5
회사 사람들하고 점심 먹으러 왔는데 음식도 맛있고 직원분들이 너무 친절하셔서 기분 좋게 배터지게 먹구 왔습니다 !!!! 특히 크림 카레 너무 부드럽고 취향저격 매일 올꺼같아요!!!!ㅎ score: 0.43333334
후기에 아르바이트생 불친절 얘기 있는데 그런 사람은 없아 구 다 친절하고 맛있고 좋았어요 score: 0.33333334
사진 찍는 인간들과 자리 차지하려고 옷가지만 남겨둔 인간들로 혼란 그 자체다. 머 이런 데가 다 그렇지. 빵과 커피도 보통. 와서 혼란스럽지만 기분 안 나쁘게 나가서 3 별. score: 0.25

price seeds (12):
굿 좋은 가격 좋은 맛 score: 0.5833334
얼그레이 케이크는 아직 못 먹어보고, 얼그레이 밀크티만 먹어봤어요 당도 13도 중 2도 먹어봤는데 생각보다 별로 인건 왜죠 밀크티가 조금 연한 거 같아서 아쉬웠네요! 그래도 브런치 세트는 가격 대비 구성이 괜찮은 것 같아요! 밀크티 포함됨 나중엔 얼그레이 케이크와 브런치 다시 먹어볼래요!! score: 0.5
고급 진 초밥 먹을 수 있는 곳! 두꺼운 네타가 올라가있어서 맛있게 먹을 수 있었다. 모둠 초밥 가격대 동네 초밥집 치고 가격대 있는 편이긴 하지만, 네타 상태 보면 비싼 가격대가 이해된다! score: 0.5
한강진 역에 위치한 카페 울프 소셜클럽. 늑대 그림의 로고가 귀엽지만 울프는 Woolf. 화장실에 쓰인 a restroom of ones own이라는 문구를 보면 더 확실해진다. 음악도 분위기도 모두 좋다. 추천   버터크림 헤븐 라테. 밀크티와 더불어 시그니처 메뉴. 달달하면서 고소한 버터 향이 은은하게 퍼져요. 요즘처럼 추운 날 잘

## 단일

In [128]:
from qdrant_client import models

def query_qdrant_hybrid(
    client,
    query_dense_emb,    # list[float] or np.ndarray
    query_sparse_emb,   # fastembed SparseEmbedding (indices, values)
    limit=5,
):
    dense_vec = list(query_dense_emb)
    sparse_vec = models.SparseVector(
        indices=list(query_sparse_emb.indices),
        values=list(query_sparse_emb.values),
    )

    res = client.query_points(
        collection_name="hybrid_test",
        query=models.FusionQuery(
            fusion=models.Fusion.RRF
        ),
        prefetch=[
            models.Prefetch(
                query=dense_vec,
                using="dense",
            ),
            models.Prefetch(
                query=sparse_vec,
                using="sparse",
            ),
        ],
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="food_category_name",
                    match=models.MatchValue(value="일식")
                )
            ]
        ),
        limit=limit,
        with_payload=True,
    )

    return res.points


In [130]:
dense_text = "이 음식점의 음식 맛, 가격/가성비, 서비스/응대에 대한 전반적인 리뷰"

In [137]:
hits_name = ["service", "price", "food"]
hits_dict = {}

for seeds, name in zip(seed_list, hits_name):

    print(f"\n{name} seeds ({len(seeds)}):")

    # dense query
    dense_q = next(dense_model.embed(" ".join(seeds[:-1])))

    # sparse query
    sparse_q = next(sparse_model.embed(" ".join(seeds[:-1])))

    hits = query_qdrant_hybrid(
        client,
        dense_q,
        sparse_q,
        limit=5
    )

    # 카테고리별 리스트 초기화
    hits_dict[name] = []

    for h in hits:
        print(h.payload["content"], "score:", h.score)

        hits_dict[name].append({
            "content": h.payload["content"],
        })
        

# hits_dict

final_seeds_dict = {
    k: [item["content"] for item in v if "content" in item]
    for k, v in hits_dict.items()
}

final_seeds_dict



service seeds (7):
회사 사람들하고 점심 먹으러 왔는데 음식도 맛있고 직원분들이 너무 친절하셔서 기분 좋게 배터지게 먹구 왔습니다 !!!! 특히 크림 카레 너무 부드럽고 취향저격 매일 올꺼같아요!!!!ㅎ score: 0.64285713
맛있고 친절하세요 score: 0.5
친절하고 맛난 음식을 먹을 수 있어서 좋아요~ 동료들과 와서 좋은 추억 만들고 갑니다! 맛도 좋아요!! score: 0.33333334
감사합니다 score: 0.25
감사합니다. score: 0.2

price seeds (12):
맛있는 다양한 종류의 덮밥 그리고 서브 메뉴로 주문한 오징어튀김까지 🦑 메뉴들이 다 맛있었어요 🐧👍🏻 덮밥 맛있는 곳이 회사 근처에 없을거라 생각했는데 너무 만족스러운 식사였습니다 스테이크 덮밥부터 항정살, 연어까지 크림카레 우삼겹은 왜 또 맛있어? 🍛 메뉴들 비주얼도 너무 훌륭했고 퀄리티들도 너무 좋았어여 💕 다음에 또 맛난 밥 먹으러 방문할게여 score: 0.5
여기 꼭 방문해 보세요. 모든 메뉴 다 먹어 볼 생각이예요. 간만에 맛집찾았어요. 너무 맛있게 먹었네요. score: 0.5
매콤한게 땡겨서 삼겹김치동 시켰습니다~ 음식 내용이 알차요^^ score: 0.5
점심에 방문예약 없이 이용대기 시간 바로 입장일상혼자 대존맛 스테키동 고기부드럽고 크림카레존맛 score: 0.33333334
혼밥만 먹다가 직장분들이랑 점심 같이 먹었는데 진짜 모든 메뉴 짱짱 다들 만족했다고 하네요^^ score: 0.33333334

food seeds (16):
연어뱃살동 맛있어요오 😋😋 감자고로케도 넘 부드럽고 맛있었음요! 사이드를 사이즈별로 시킬수있게 해두어서 혼밥항 때도 사이드 먹기 좋네요 ㅎㅎ 유스페이스 맛집입니당 score: 0.5
회사 사람들하고 점심 먹으러 왔는데 음식도 맛있고 직원분들이 너무 친절하셔서 기분 좋게 배터지게 먹구 왔습니다 !!!! 특히 크림 카레 너무 부드럽고 취향저격 매일 올꺼같아요!!!!ㅎ score: 0.5
맛있어요. 페이

{'service': ['회사 사람들하고 점심 먹으러 왔는데 음식도 맛있고 직원분들이 너무 친절하셔서 기분 좋게 배터지게 먹구 왔습니다 !!!! 특히 크림 카레 너무 부드럽고 취향저격 매일 올꺼같아요!!!!ㅎ',
  '맛있고 친절하세요',
  '친절하고 맛난 음식을 먹을 수 있어서 좋아요~ 동료들과 와서 좋은 추억 만들고 갑니다! 맛도 좋아요!!',
  '감사합니다',
  '감사합니다.'],
 'price': ['맛있는 다양한 종류의 덮밥 그리고 서브 메뉴로 주문한 오징어튀김까지 🦑 메뉴들이 다 맛있었어요 🐧👍🏻 덮밥 맛있는 곳이 회사 근처에 없을거라 생각했는데 너무 만족스러운 식사였습니다 스테이크 덮밥부터 항정살, 연어까지 크림카레 우삼겹은 왜 또 맛있어? 🍛 메뉴들 비주얼도 너무 훌륭했고 퀄리티들도 너무 좋았어여 💕 다음에 또 맛난 밥 먹으러 방문할게여',
  '여기 꼭 방문해 보세요. 모든 메뉴 다 먹어 볼 생각이예요. 간만에 맛집찾았어요. 너무 맛있게 먹었네요.',
  '매콤한게 땡겨서 삼겹김치동 시켰습니다~ 음식 내용이 알차요^^',
  '점심에 방문예약 없이 이용대기 시간 바로 입장일상혼자 대존맛 스테키동 고기부드럽고 크림카레존맛',
  '혼밥만 먹다가 직장분들이랑 점심 같이 먹었는데 진짜 모든 메뉴 짱짱 다들 만족했다고 하네요^^'],
 'food': ['연어뱃살동 맛있어요오 😋😋 감자고로케도 넘 부드럽고 맛있었음요! 사이드를 사이즈별로 시킬수있게 해두어서 혼밥항 때도 사이드 먹기 좋네요 ㅎㅎ 유스페이스 맛집입니당',
  '회사 사람들하고 점심 먹으러 왔는데 음식도 맛있고 직원분들이 너무 친절하셔서 기분 좋게 배터지게 먹구 왔습니다 !!!! 특히 크림 카레 너무 부드럽고 취향저격 매일 올꺼같아요!!!!ㅎ',
  '맛있어요. 페이코 식권 해주세요 :)',
  '점심식사로도 좋고 퇴근후 간단히 술한잔 하기도 좋은 메뉴들 맛있게 잘 먹었습니다~',
  '사케동, 연어뱃살동의 연어가 신선하고 맛있습니다. 고로케 바삭하고 고소해요.']}

In [ ]:
import os, json, re
from openai import OpenAI

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

PRICE_HINTS = [
    "가격", "가성비", "저렴", "비싸", "비쌈", "가격대", "합리", "구성", "구성비",
    "양", "푸짐", "리필", "무한", "만족", "혜자"
]

def _clip(xs, n=8):
    xs = [x.strip() for x in xs if x and str(x).strip()]
    return xs[:n]

def _has_price_signal(text: str) -> bool:
    t = text.replace(" ", "")
    return any(k.replace(" ", "") in t for k in PRICE_HINTS)

def summarize_aspects(
    service_reviews: list[str],
    price_reviews: list[str],
    food_reviews: list[str],
    model: str = "gpt-5",
    per_category_max: int = 8,
) -> dict:

    payload = {
        "service": _clip(service_reviews, per_category_max),
        "price": _clip(price_reviews, per_category_max),
        "food": _clip(food_reviews, per_category_max),
    }

    instructions = """
너는 음식점 리뷰 분석가다.
입력으로 카테고리별 '근거 리뷰 목록'이 주어진다.
아래 JSON 스키마로만 출력하라(추가 텍스트 금지).

스키마:
{
  "service": {"summary": string, "bullets": [string, ...], "evidence": [int, ...]},
  "price":   {"summary": string, "bullets": [string, ...], "evidence": [int, ...]},
  "food":    {"summary": string, "bullets": [string, ...], "evidence": [int, ...]},
  "overall_summary": {"summary": string}
}

규칙:
- 각 카테고리 summary: 1문장, 과장 금지
- bullets: 3~5개, 중복 제거, 구체적으로
- evidence: 근거로 쓴 리뷰의 인덱스(각 카테고리 리스트에서 0-based)
- price는 '가격 숫자'가 없으면 '가성비/양/구성/만족감' 같은 우회표현을 근거로 요약하라.
- overall_summary는 2~3문장으로 종합 요약하라.
- 근거(입력 리뷰)에 없는 내용은 추측하지 말고 "언급이 적다"라고 표현하라.
"""

    def call():
        resp = client.responses.create(
            model=model,
            reasoning={"effort": "low"},
            instructions=instructions,
            input=json.dumps(payload, ensure_ascii=False),
        )
        return resp.output_text.strip()

    # 1) 1차 호출
    text = call()

    # 2) JSON 파싱 + 실패 시 1회 재시도
    try:
        out = json.loads(text)
    except json.JSONDecodeError:
        # JSON만 다시 출력하도록 한 번 더 강제
        fix = client.responses.create(
            model=model,
            reasoning={"effort": "low"},
            instructions="다음 텍스트를 '유효한 JSON'으로만 변환해. 다른 말 금지.",
            input=text,
        ).output_text.strip()
        out = json.loads(fix)

    # 3) evidence 인덱스 검증 (범위 밖이면 제거)
    for cat in ("service", "price", "food"):
        n = len(payload[cat])
        ev = out.get(cat, {}).get("evidence", [])
        if isinstance(ev, list):
            out[cat]["evidence"] = [i for i in ev if isinstance(i, int) and 0 <= i < n]
        else:
            out[cat]["evidence"] = []

    # 4) price 요약 게이트: evidence 리뷰들에 price 신호가 전혀 없으면 안전하게 다운그레이드
    price_ev = out.get("price", {}).get("evidence", [])
    ev_texts = [payload["price"][i] for i in price_ev] if price_ev else []
    if ev_texts and not any(_has_price_signal(t) for t in ev_texts):
        out["price"]["summary"] = "가격 관련 언급이 많지 않아, 전반적인 만족감/구성(양 등) 중심으로만 해석 가능합니다."
        # bullets도 너무 단정적으로 쓰지 않게 최소화
        out["price"]["bullets"] = [
            "가격을 직접 언급한 리뷰가 많지 않습니다.",
            "대신 만족/구성/양(푸짐함) 관련 표현이 간접적으로 나타납니다."
        ]

    return out


if __name__ == "__main__":
    service = final_seeds_dict["service"]
    price = final_seeds_dict["price"]
    food = final_seeds_dict["food"]

    out = summarize_aspects(service, price, food)
    print(json.dumps(out, ensure_ascii=False, indent=2))


{
  "service": {
    "summary": "직원들이 친절하고 동료들과의 점심 방문에도 응대가 좋아 기분 좋은 식사가 가능했다.",
    "bullets": [
      "점심에 방문했을 때 직원들이 매우 친절해 기분 좋게 식사했다",
      "동료들과 함께 방문해 친절한 응대 덕분에 좋은 추억을 만들었다",
      "전반적으로 맛과 함께 친절한 서비스가 인상적이었다"
    ],
    "evidence": [
      0,
      2,
      1
    ]
  },
  "price": {
    "summary": "다양한 메뉴 구성과 알찬 내용물, 좋은 퀄리티로 전반적인 만족도가 높은 가성비를 보인다.",
    "bullets": [
      "덮밥부터 튀김까지 메뉴가 다양하고 퀄리티가 좋아 만족스러운 식사였다",
      "삼겹김치동의 구성과 내용물이 알차다고 평가됐다",
      "점심 시간에 대기 없이 바로 입장해 이용 효율이 좋았다",
      "여러 명이 함께 먹고 모두 만족했다고 평가했다"
    ],
    "evidence": [
      0,
      2,
      3,
      4
    ]
  },
  "food": {
    "summary": "연어동과 크림 카레 등 주요 메뉴가 신선하고 부드러우며, 사이드 메뉴까지 완성도가 높다.",
    "bullets": [
      "연어뱃살동과 사케동의 연어가 신선하고 맛이 좋다",
      "크림 카레가 매우 부드럽고 취향을 저격했다",
      "감자고로케가 바삭하면서도 고소하고 식감이 좋다",
      "사이드 메뉴를 사이즈별로 선택할 수 있어 혼밥에도 적합하다",
      "점심은 물론 퇴근 후 가볍게 곁들이기에도 어울리는 메뉴 구성이다"
    ],
    "evidence": [
      0,
      4,
      1,
      0,
      3
    ]
  },
  "overall_summary": {
 

{
  "service": {
    "summary": "직원들이 친절하고 동료들과의 점심 방문에도 응대가 좋아 기분 좋은 식사가 가능했다.",
    "bullets": [
      "점심에 방문했을 때 직원들이 매우 친절해 기분 좋게 식사했다",
      "동료들과 함께 방문해 친절한 응대 덕분에 좋은 추억을 만들었다",
      "전반적으로 맛과 함께 친절한 서비스가 인상적이었다"
    ],
    "evidence": [
      0,
      2,
      1
    ]
  },
  "price": {
    "summary": "다양한 메뉴 구성과 알찬 내용물, 좋은 퀄리티로 전반적인 만족도가 높은 가성비를 보인다.",
    "bullets": [
      "덮밥부터 튀김까지 메뉴가 다양하고 퀄리티가 좋아 만족스러운 식사였다",
      "삼겹김치동의 구성과 내용물이 알차다고 평가됐다",
      "점심 시간에 대기 없이 바로 입장해 이용 효율이 좋았다",
      "여러 명이 함께 먹고 모두 만족했다고 평가했다"
    ],
    "evidence": [
      0,
      2,
      3,
      4
    ]
  },
  "food": {
    "summary": "연어동과 크림 카레 등 주요 메뉴가 신선하고 부드러우며, 사이드 메뉴까지 완성도가 높다.",
    "bullets": [
      "연어뱃살동과 사케동의 연어가 신선하고 맛이 좋다",
      "크림 카레가 매우 부드럽고 취향을 저격했다",
      "감자고로케가 바삭하면서도 고소하고 식감이 좋다",
      "사이드 메뉴를 사이즈별로 선택할 수 있어 혼밥에도 적합하다",
      "점심은 물론 퇴근 후 가볍게 곁들이기에도 어울리는 메뉴 구성이다"
    ],
    "evidence": [
      0,
      4,
      1,
      0,
      3
    ]
  },
  "overall_summary": {
    "overall_summary": "친절한 응대와 쾌적한 이용 경험 덕분에 동료들과의 식사 장소로 만족도가 높다. 다양한 덮밥과 사이드가 신선하고 완성도가 높아 전반적으로 가성비 좋은 한 끼로 평가된다."
  }
}

In [141]:
import os, json, re
from openai import OpenAI

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

PRICE_HINTS = [
    "가격", "가성비", "저렴", "비싸", "비쌈", "가격대", "합리", "구성", "구성비",
    "양", "푸짐", "리필", "무한", "만족", "혜자"
]

def _clip(xs, n=8):
    xs = [x.strip() for x in xs if x and str(x).strip()]
    return xs[:n]

def _has_price_signal(text: str) -> bool:
    t = text.replace(" ", "")
    return any(k.replace(" ", "") in t for k in PRICE_HINTS)

def summarize_aspects(
    service_reviews: list[str],
    price_reviews: list[str],
    food_reviews: list[str],
    model: str = "gpt-5",
    per_category_max: int = 8,
) -> dict:

    payload = {
        "service": _clip(service_reviews, per_category_max),
        "price": _clip(price_reviews, per_category_max),
        "food": _clip(food_reviews, per_category_max),
    }

    instructions = """
너는 음식점 리뷰 분석가다.
입력으로 카테고리별 '근거 리뷰 목록'이 주어진다.
아래 JSON 스키마로만 출력하라(추가 텍스트 금지).

스키마:
{
  "service": {"summary": string, "bullets": [string, ...], "evidence": [int, ...]},
  "price":   {"summary": string, "bullets": [string, ...], "evidence": [int, ...]},
  "food":    {"summary": string, "bullets": [string, ...], "evidence": [int, ...]},
  "overall_summary": {"summary": string}
}

규칙:
- 각 카테고리 summary: 1문장, 과장 금지
- bullets: 3~5개, 중복 제거, 구체적으로
- evidence: 근거로 쓴 리뷰의 인덱스(각 카테고리 리스트에서 0-based)
- price는 '가격 숫자'가 없으면 '가성비/양/구성/만족감' 같은 우회표현을 근거로 요약하라.
- overall_summary는 2~3문장으로 종합 요약하라.
- 근거(입력 리뷰)에 없는 내용은 추측하지 말고 "언급이 적다"라고 표현하라.
"""

    def call():
        resp = client.responses.create(
            model=model,
            reasoning={"effort": "low"},
            instructions=instructions,
            input=json.dumps(payload, ensure_ascii=False),
        )
        return resp.output_text.strip()

    # 1) 1차 호출
    text = call()

    # 2) JSON 파싱 + 실패 시 1회 재시도
    try:
        out = json.loads(text)
    except json.JSONDecodeError:
        # JSON만 다시 출력하도록 한 번 더 강제
        fix = client.responses.create(
            model=model,
            reasoning={"effort": "low"},
            instructions="다음 텍스트를 '유효한 JSON'으로만 변환해. 다른 말 금지.",
            input=text,
        ).output_text.strip()
        out = json.loads(fix)

    # 3) evidence 인덱스 검증 (범위 밖이면 제거)
    for cat in ("service", "price", "food"):
        n = len(payload[cat])
        ev = out.get(cat, {}).get("evidence", [])
        if isinstance(ev, list):
            out[cat]["evidence"] = [i for i in ev if isinstance(i, int) and 0 <= i < n]
        else:
            out[cat]["evidence"] = []

    # 4) price 요약 게이트: evidence 리뷰들에 price 신호가 전혀 없으면 안전하게 다운그레이드
    price_ev = out.get("price", {}).get("evidence", [])
    ev_texts = [payload["price"][i] for i in price_ev] if price_ev else []
    if ev_texts and not any(_has_price_signal(t) for t in ev_texts):
        out["price"]["summary"] = "가격 관련 언급이 많지 않아, 전반적인 만족감/구성(양 등) 중심으로만 해석 가능합니다."
        # bullets도 너무 단정적으로 쓰지 않게 최소화
        out["price"]["bullets"] = [
            "가격을 직접 언급한 리뷰가 많지 않습니다.",
            "대신 만족/구성/양(푸짐함) 관련 표현이 간접적으로 나타납니다."
        ]

    return out


if __name__ == "__main__":
    service = final_seeds_dict["service"]
    price = final_seeds_dict["price"]
    food = final_seeds_dict["food"]

    out = summarize_aspects(service, price, food)
    print(json.dumps(out, ensure_ascii=False, indent=2))


{
  "service": {
    "summary": "직원들이 전반적으로 친절하다는 후기가 많아 동료들과 기분 좋게 식사하기에 적합하지만, 응대 속도나 대기 관련 구체적 언급은 적다.",
    "bullets": [
      "직원 응대가 친절했다",
      "회사 동료들과의 방문에 만족했고 좋은 추억을 남겼다",
      "식사 내내 기분 좋았다는 후기가 있다",
      "감사 인사를 남긴 리뷰가 다수다"
    ],
    "evidence": [
      0,
      1,
      2,
      3,
      4
    ]
  },
  "price": {
    "summary": "구체적 금액 언급은 없지만 메뉴 구성이 알차고 품질·비주얼 만족도가 높아 전반적으로 가성비가 좋다는 인상이며, 가격 자체에 대한 언급은 적다.",
    "bullets": [
      "덮밥 종류가 다양하고 오징어튀김 등 사이드까지 구성에 대한 만족도가 높다",
      "메뉴 비주얼과 퀄리티가 좋다는 평가",
      "삼겹김치동이 내용이 알차다는 의견",
      "여러 메뉴를 다시 먹고 싶을 만큼 만족했다"
    ],
    "evidence": [
      0,
      2,
      1,
      4
    ]
  },
  "food": {
    "summary": "연어덮밥류의 신선도와 크림카레, 고로케 등 사이드의 맛이 호평을 받았고 전반적으로 맛있다는 평가가 많다.",
    "bullets": [
      "연어뱃살동·사케동의 연어가 신선하고 맛있다는 평가",
      "크림 카레가 부드럽고 취향에 맞았다는 후기",
      "감자고로케가 부드럽고 바삭하며 고소하다는 의견",
      "전반적으로 메뉴가 맛있었다는 반응"
    ],
    "evidence": [
      4,
      1,
      0,
      2,
      3
    ]
  },
  "overall_summary": {
 

{
  "service": {
    "summary": "직원들이 전반적으로 친절하다는 후기가 많아 동료들과 기분 좋게 식사하기에 적합하지만, 응대 속도나 대기 관련 구체적 언급은 적다.",
    "bullets": [
      "직원 응대가 친절했다",
      "회사 동료들과의 방문에 만족했고 좋은 추억을 남겼다",
      "식사 내내 기분 좋았다는 후기가 있다",
      "감사 인사를 남긴 리뷰가 다수다"
    ],
    "evidence": [
      0,
      1,
      2,
      3,
      4
    ]
  },
  "price": {
    "summary": "구체적 금액 언급은 없지만 메뉴 구성이 알차고 품질·비주얼 만족도가 높아 전반적으로 가성비가 좋다는 인상이며, 가격 자체에 대한 언급은 적다.",
    "bullets": [
      "덮밥 종류가 다양하고 오징어튀김 등 사이드까지 구성에 대한 만족도가 높다",
      "메뉴 비주얼과 퀄리티가 좋다는 평가",
      "삼겹김치동이 내용이 알차다는 의견",
      "여러 메뉴를 다시 먹고 싶을 만큼 만족했다"
    ],
    "evidence": [
      0,
      2,
      1,
      4
    ]
  },
  "food": {
    "summary": "연어덮밥류의 신선도와 크림카레, 고로케 등 사이드의 맛이 호평을 받았고 전반적으로 맛있다는 평가가 많다.",
    "bullets": [
      "연어뱃살동·사케동의 연어가 신선하고 맛있다는 평가",
      "크림 카레가 부드럽고 취향에 맞았다는 후기",
      "감자고로케가 부드럽고 바삭하며 고소하다는 의견",
      "전반적으로 메뉴가 맛있었다는 반응"
    ],
    "evidence": [
      4,
      1,
      0,
      2,
      3
    ]
  },
  "overall_summary": {
    "summary": "친절한 응대와 함께 연어덮밥, 크림카레, 고로케 등 음식의 맛과 퀄리티에 대한 호평이 많다. 가격 숫자 정보는 없지만 구성과 만족도가 높아 가성비가 좋다는 인상이며, 세부적인 가격 정보는 언급이 적다."
  }
}

In [117]:
# 전처리 후, 앞뒤 같은 단어 제거, 카테고리에 어울리지 않는 단어 제거
service_seeds = ['직원 친절', '사장 친절', '친절 기분', '서비스 친절', '사장 직원', '직원 서비스', '아주머니 친절']
price_seeds = ['가격 대비', '무한 리필', '가격 생각', '음식 가격', '합리 가격', '메뉴 가격', '가격 만족', '가격 퀄리티', '리필 가능', '커피 가격', '가격 사악', '런치 가격']
food_seeds = ['가락 국수', '평양 냉면', '수제 버거', '크림 치즈', '치즈 케이크', '크림 파스타', '당근 케이크', '오일 파스타', '카페 커피', '비빔 냉면', '커피 원두', '리코타 치즈', '비빔 막국수', '치즈 돈가스', '커피 산미', '치즈 파스타']

seed_list = [service_seeds, price_seeds, food_seeds]
name_list = ["service", "price", "food"]

In [118]:
for seeds, name in zip(seed_list, name_list):

    print(f"\n{name} seeds ({len(seeds)}):")

    # dense query
    dense_q = next(dense_model.embed(" ".join(seeds[:-1])))

    # sparse query
    sparse_q = next(sparse_model.embed(" ".join(seeds[:-1])))

    hits = query_qdrant_hybrid(
        client,
        dense_q,
        sparse_q,
        limit=5
    )

    for h in hits:
        print(h.payload["content"], "score:", h.score)


service seeds (7):
신규 개업했을 때 궁금해서 가봤어요 오며 가며 이 골목 많이 지나다니는데 새로 개업한 곳임에도 줄이 엄청 길더군요 평일 저녁 한 시간 안되게 웨이팅 해서 들어갔어요,, 정가를 비랑 이동갈비 먹었고, 육회비빔밥도 맛봤는데 다 무난하게 맛있더라고요! 양념도 지나치게 달거나 짜지 않고 적당해요  무엇보다 소갈비치고, 그것도 강남권에서 이 가격이면 저렴한 편이니까 기분 좋게 방문할 만한 곳이라는 주관적인 판단,, 개인적으로는 길게 웨이팅을 해가면서 찾아와 먹을 정도까지는 아니어도 근처 사시는 분들이나, 웨이팅 없을 때 소갈비생각 나시면 들러볼 만한 곳입니다 score: 0.64285713
후기에 아르바이트생 불친절 얘기 있는데 그런 사람은 없아 구 다 친절하고 맛있고 좋았어요 score: 0.5
정말 맛있긴 한데 매니저분 너무 불친절함... score: 0.33333334
항상 손님이 많고 정신없지만 자주 떠오르는 누룽지 백숙. 방문할 때마다 맛있게 잘 먹었다는 만족감을 줍니다. score: 0.25
비주얼부터 심쿵    보는 그대로의 감동이 입에도 전해져요 훕 score: 0.2

price seeds (12):
신규 개업했을 때 궁금해서 가봤어요 오며 가며 이 골목 많이 지나다니는데 새로 개업한 곳임에도 줄이 엄청 길더군요 평일 저녁 한 시간 안되게 웨이팅 해서 들어갔어요,, 정가를 비랑 이동갈비 먹었고, 육회비빔밥도 맛봤는데 다 무난하게 맛있더라고요! 양념도 지나치게 달거나 짜지 않고 적당해요  무엇보다 소갈비치고, 그것도 강남권에서 이 가격이면 저렴한 편이니까 기분 좋게 방문할 만한 곳이라는 주관적인 판단,, 개인적으로는 길게 웨이팅을 해가면서 찾아와 먹을 정도까지는 아니어도 근처 사시는 분들이나, 웨이팅 없을 때 소갈비생각 나시면 들러볼 만한 곳입니다 score: 0.5
카사블랑카를 워낙 좋아하는데 망고밍글링에서 크리스틴님으로부터 똑같은 사장님이 맞은편에 모 로코코를 오픈하셨다는 얘길 듣고 벼르다가 방문했다! 카사블랑카가 

In [22]:
hits = query_qdrant_hybrid(client,"든든한 한 끼")

for hit in hits:
    print(hit.payload["content"], "score:", hit.score)

11시 오픈이래서 11시 5분에 가니 벌써 만석. 세 명이서 꼬막 비빔밥 하나에 밥 한 공기 추가해 배부르게 먹었다. 먹다 보니 양이 많아서 밥 하나는 안 시켜도 될뻔했다. 밥이 비벼 친 채로 반  꼬막만 반 나오는데 밥을 비빈 부분이 안 비빈 부분보다 확실히 싱겁다. 짜게 먹을 사람들은 밥에 꼬막을 더 얹어 먹으면 된다. 재료에 참기름만 있는 건 아닌 것 같고 아무래도 들기름도 같이 넣은 듯. 꼬막에 윤기가 좌르르 흐른다. 꼬막이 통통하고 살이 잘 차올라서 쫄깃쫄깃 씹는 맛이 있다. 게다가 파와 고추가 튼실하고 양이 많아서 비주얼 적으로도, 맛으로도 만족스러웠다. 양념이 되어있고 밥이랑 같이 비벼 먹다 보니 마지막에 좀 더부룩하긴 하였지만 강릉 와서 한 번쯤 가보면 좋을 곳. score: 0.75
비주얼부터 심쿵    보는 그대로의 감동이 입에도 전해져요 훕 score: 0.5
신규 개업했을 때 궁금해서 가봤어요 오며 가며 이 골목 많이 지나다니는데 새로 개업한 곳임에도 줄이 엄청 길더군요 평일 저녁 한 시간 안되게 웨이팅 해서 들어갔어요,, 정가를 비랑 이동갈비 먹었고, 육회비빔밥도 맛봤는데 다 무난하게 맛있더라고요! 양념도 지나치게 달거나 짜지 않고 적당해요  무엇보다 소갈비치고, 그것도 강남권에서 이 가격이면 저렴한 편이니까 기분 좋게 방문할 만한 곳이라는 주관적인 판단,, 개인적으로는 길게 웨이팅을 해가면서 찾아와 먹을 정도까지는 아니어도 근처 사시는 분들이나, 웨이팅 없을 때 소갈비생각 나시면 들러볼 만한 곳입니다 score: 0.33333334
앙버터가 세젤맛입니당 score: 0.33333334
아이들이 재밌게 놀 수도 있어요 score: 0.2


## Dense

In [23]:
from qdrant_client import QdrantClient, models
from qdrant_client.models import VectorParams, Distance, SparseVectorParams, PointStruct, SparseVector

def setup_qdrant_dense(passages, dense_embeddings, sparse_embeddings):
    client = QdrantClient('http://localhost:6333')
    collection_name = "dense_test"
    dense_dim = len(dense_embeddings[0])
    
    if client.collection_exists(collection_name):
        client.delete_collection(collection_name)
        
    client.create_collection(
        collection_name = collection_name,
        vectors_config = {
            "dense" : VectorParams(
                size = dense_dim,
                distance = Distance.COSINE
            )
        },
        #sparse_vectors_config = {
            #"sparse" : SparseVectorParams()
        #}
    )
    
    points = []
    for i, (passage, dense_emb, sparse_emb) in enumerate(zip(passages, dense_embeddings, sparse_embeddings)):
        
        dense_vector = list(dense_emb)
    
        #sparse_vector = SparseVector(
        #    indices = list(sparse_emb.indices),
        #    values = list(sparse_emb.values)
        #)
        
        point = PointStruct(
            id = i,
            vector = {
                "dense" : dense_vector,
                #"sparse" : sparse_vector
            },
            payload = passages[i] # 딕서너리 -> qdrant 가능, 딕셔너리 <-> 판다스 df, 판다스 df -> qdrant payload 가능
        )
        points.append(point)
        
    client.upsert(collection_name=collection_name, points=points)
    return client

In [24]:
client = setup_qdrant_dense(dataset, dense_embeddings, sparse_embeddings)

In [ ]:
from qdrant_client import models

def query_qdrant_dense_only(client, query_dense_emb, limit=5):
    query_dense = list(query_dense_emb)  # numpy/torch -> list[float]

    res = client.query_points(
        collection_name="dense_test",  # 또는 hybrid 컬렉션 이름
        query=query_dense,             # ✅ dense vector 직접 전달
        using="dense",                 # ✅ dense named vector
        limit=limit,
        with_payload=True,
    )
    return res.points


In [ ]:
q = next(dense_model.embed(["든든"]))  # fastembed dense embedding
hits = query_qdrant_dense_only(client, q, limit=5)

for i in hits:
    print(i.payload["content"], "score:", i.score)

In [ ]:
'''
def query_qdrant_dense(client,query_text,limit=5):
    res = client.query_points(
        collection_name = "dense_test",
            query=models.Document(
                text=query_text, 
                model="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
            ),
            using="dense",
            query_filter=models.Filter(
                must=[
                    models.FieldCondition(
                        key="food_category_name",
                        match=models.MatchValue(value="중식")
                    )
                ]
            ),
        limit=limit,
        with_payload=True
    )
    
    return res.points
'''

In [ ]:
'''
hits = query_qdrant_dense(client,"국밥", limit=1)

for hit in hits:
    print(hit.payload["content"], "score:", hit.score)
'''

11시 오픈이래서 11시 5분에 가니 벌써 만석. 세 명이서 꼬막 비빔밥 하나에 밥 한 공기 추가해 배부르게 먹었다. 먹다 보니 양이 많아서 밥 하나는 안 시켜도 될뻔했다. 밥이 비벼 친 채로 반  꼬막만 반 나오는데 밥을 비빈 부분이 안 비빈 부분보다 확실히 싱겁다. 짜게 먹을 사람들은 밥에 꼬막을 더 얹어 먹으면 된다. 재료에 참기름만 있는 건 아닌 것 같고 아무래도 들기름도 같이 넣은 듯. 꼬막에 윤기가 좌르르 흐른다. 꼬막이 통통하고 살이 잘 차올라서 쫄깃쫄깃 씹는 맛이 있다. 게다가 파와 고추가 튼실하고 양이 많아서 비주얼 적으로도, 맛으로도 만족스러웠다. 양념이 되어있고 밥이랑 같이 비벼 먹다 보니 마지막에 좀 더부룩하긴 하였지만 강릉 와서 한 번쯤 가보면 좋을 곳. score: 0.71336967


## Sparse

In [58]:
from qdrant_client import QdrantClient, models
from qdrant_client.models import VectorParams, Distance, SparseVectorParams, PointStruct, SparseVector

def setup_qdrant_sparse(passages, dense_embeddings, sparse_embeddings):
    client = QdrantClient('http://localhost:6333')
    collection_name = "sparse_test"
    #dense_dim = len(dense_embeddings[0])
    
    if client.collection_exists(collection_name):
        client.delete_collection(collection_name)
        
    client.create_collection(
        collection_name = collection_name,
        #vectors_config = {
        #    "dense" : VectorParams(
        #        size = dense_dim,
        #        distance = Distance.COSINE
        #    )
        #},
        sparse_vectors_config = {
            "sparse" : SparseVectorParams()
        }
    )
    
    points = []
    for i, (passage, dense_emb, sparse_emb) in enumerate(zip(passages, dense_embeddings, sparse_embeddings)):
        
        #dense_vector = list(dense_emb)
    
        sparse_vector = SparseVector(
            indices = list(sparse_emb.indices),
            values = list(sparse_emb.values)
        )
        
        point = PointStruct(
            id = i,
            vector = {
                #"dense" : dense_vector,
                "sparse" : sparse_vector
            },
            payload = passages[i] # 딕서너리 -> qdrant 가능, 딕셔너리 <-> 판다스 df, 판다스 df -> qdrant payload 가능
        )
        points.append(point)
        
    client.upsert(collection_name=collection_name, points=points)
    return client

In [ ]:
client = setup_qdrant_sparse(dataset, dense_embeddings, sparse_embeddings)

In [76]:
def query_qdrant_sparse(client,query_text,limit=5):
    res = client.query_points(
        collection_name = "sparse_test",
            query=models.Document(
                text=query_text, 
                model="Qdrant/bm25"
            ),
            using="sparse",
            query_filter=models.Filter(
                must=[
                    models.FieldCondition(
                        key="food_category_name",
                        match=models.MatchValue(value="중식")
                    )
                ]
            ),
        limit=limit,
        with_payload=True
    )
    
    return res.points

In [77]:
hits = query_qdrant_sparse(client,"임진각")

for hit in hits:
    print(hit.payload["content"], "score:", hit.score)

임진각 가족여행 왔다가 들렸어요 전방에 있는 부대찌개집이어서 그런지 전통적인 맛이 나네요 score: 1.6434199


In [63]:
from qdrant_client import models

def query_qdrant_sparse_only(client, query_sparse_emb, limit=5):
    # query_sparse_emb 가 (indices, values)를 가진 객체라고 가정
    query_sparse = models.SparseVector(
        indices=list(query_sparse_emb.indices),
        values=list(query_sparse_emb.values),
    )
 
    res = client.query_points(
        collection_name="sparse_test",
        query=query_sparse,
        using="sparse",          # ✅ sparse named vector
        limit=limit,
        with_payload=True,
    )
    return res.points


In [75]:
q_sparse = next(sparse_model.embed(["임진각"]))
hits = query_qdrant_sparse_only(client, q_sparse, limit=1)

for h in hits:
    print(h.payload.get("content"), "score:", h.score)


임진각 가족여행 왔다가 들렸어요 전방에 있는 부대찌개집이어서 그런지 전통적인 맛이 나네요 score: 2.7736712


결론

1. Dense는 고정 차원이 필요, sparse는 고정차원이 없음.
2. next는 generator에서 1개만 꺼내는 함수
3. sparse는 값이 안나와서, qdrant/bm25를 fastembed의 document 객체가 변환하지 못하는걸로 착각. 사실은 그냥 결과 값이 없었음.(문서에 해당 쿼리에 해당하는 값 없음)